In [ ]:
pip install sciann

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 168 kB 4.1 MB/s 
     |████████████████████████████████| 561 kB 45.6 MB/s 
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=99cc86f0389d1c93408f43c2e6e583ad65172aab8bd46e469904b71dec3c6534
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [ ]:
!git clone https://github.com/ianbgroves/sciann-applications

Cloning into 'sciann-applications'...
remote: Enumerating objects: 815, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 815 (delta 49), reused 193 (delta 46), pack-reused 594
Receiving objects: 100% (815/815), 492.24 MiB | 15.31 MiB/s, done.
Resolving deltas: 100% (72/72), done.
Checking out files: 100% (676/676), done.


In [ ]:
import numpy as np
import sciann as sn
import matplotlib.pyplot as plt
import scipy.io
import os

---------------------- SCIANN 0.6.7.6 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 
TensorFlow Version: 2.8.2 
Python Version: 3.7.14 (default, Sep  8 2022, 00:06:44) 
[GCC 7.5.0] 



In [ ]:
# "." for Colab/VSCode, and ".." for GitHub
repoPath = os.path.join(".", "PINNs")
# repoPath = os.path.join("..", "PINNs")
utilsPath = os.path.join(repoPath, "Utilities")
dataPath = os.path.join(repoPath, "main", "Data")
appDataPath = os.path.join(repoPath, "appendix", "Data")


In [ ]:
ls

sample_data/  sciann-applications/


# Introduction
This is the code for the Navier-Stokes inversion problem from SciANN paper:
+ https://arxiv.org/abs/2005.08803
+ https://www.sciencedirect.com/science/article/pii/S0045782520307374

The training data can be found in the following repository:
+ https://github.com/maziarraissi/PINNs/tree/master/main/Data/cylinder_nektar_wake.mat

In [ ]:
def PrepareData(num_data=5000, random=True):

    # Get data file from:
    #         https://github.com/maziarraissi/PINNs/tree/master/main/Data/cylinder_nektar_wake.mat
    data = scipy.io.loadmat('sciann-applications/PINNs/main/Data/cylinder_nektar_wake.mat')

    U_star = data['U_star'] # N x 2 x T
    P_star = data['p_star'] # N x T
    t_star = data['t'] # T x 1
    X_star = data['X_star'] # N x 2

    N = X_star.shape[0]
    T = t_star.shape[0]

    # Rearrange Data
    XX = np.tile(X_star[:,0:1], (1,T)) # N x T
    YY = np.tile(X_star[:,1:2], (1,T)) # N x T
    TT = np.tile(t_star, (1,N)).T # N x T

    UU = U_star[:,0,:] # N x T
    VV = U_star[:,1,:] # N x T
    PP = P_star # N x T

    # Pick random data.
    if random:
        idx = np.random.choice(N*T, num_data, replace=False)
    else:
        idx = np.arange(0, N*T)

    x = XX.flatten()[idx,None] # NT x 1
    y = YY.flatten()[idx,None] # NT x 1
    t = TT.flatten()[idx,None] # NT x 1

    u = UU.flatten()[idx,None] # NT x 1
    v = VV.flatten()[idx,None] # NT x 1
    p = PP.flatten()[idx,None] # NT x 1

    return (x,y,t,u,v,p)

In [ ]:
x_train, y_train, t_train, u_train, v_train, p_train = PrepareData(5000, random=True)

# PINN setup

As discussed in the paper, the independent variables are $(x,y,t)$ and the solution variables are $(p, \psi)$. The velocities are defined as $u=\psi_{,y}$ and $v=-\psi_{,x}$.

The neural networks are:
$$
p: (x,y,t) \rightarrow \mathcal{N}_p(x,y,t; \mathbf{W}, \mathbf{b})
\psi: (x,y,t) \rightarrow \mathcal{N}_{\psi}(x,y,t; \mathbf{W}, \mathbf{b})
$$


The governing relations are summerized as:
$$
u_{,t} + p_{,x} + \lambda_1 (u u_{,x} + v u_{,y}) - \lambda_2 (u_{,xx} + u_{,yy}) = 0 \\
v_{,t} + p_{,y} + \lambda_1 (u v_{,x} + v v_{,y}) - \lambda_2 (v_{,xx} + v_{,yy}) = 0
$$



Define independent variables with `sn.Variable`:

In [ ]:
x = sn.Variable("x", dtype='float64')
y = sn.Variable("y", dtype='float64')
t = sn.Variable("t", dtype='float64')

Define solution variables with `sn.Functional` (multi-layer neural network):

In [ ]:
P = sn.Functional("P", [x, y, t], 8*[20], 'tanh')
Psi = sn.Functional("Psi", [x, y, t], 8*[20], 'tanh')

For inversion, define parameters using `sn.Parameter`:

In [ ]:
lambda1 = sn.Parameter(np.random.rand(), inputs=[x,y,t], name="lambda1")
lambda2 = sn.Parameter(np.random.rand(), inputs=[x,y,t], name="lambda2")

Use `sn.diff` and other mathematical operations to set up the PINN model.

In [ ]:
u = sn.diff(Psi, y)
v = -sn.diff(Psi, x)

u_t = sn.diff(u, t)
u_x = sn.diff(u, x)
u_y = sn.diff(u, y)
u_xx = sn.diff(u, x, order=2)
u_yy = sn.diff(u, y, order=2)

v_t = sn.diff(v, t)
v_x = sn.diff(v, x)
v_y = sn.diff(v, y)
v_xx = sn.diff(v, x, order=2)
v_yy = sn.diff(v, y, order=2)

p_x = sn.diff(P, x)
p_y = sn.diff(P, y)

Define targets (losses) using `sn.Data`, `sn.Tie`, and `sn.PDE` interfaces.

In [ ]:
# Define constraints
d1 = sn.Data(u)
d2 = sn.Data(v)
d3 = sn.Data(P)

c1 = sn.Tie(-p_x, u_t+lambda1*(u*u_x+v*u_y)-lambda2*(u_xx+u_yy))
c2 = sn.Tie(-p_y, v_t+lambda1*(u*v_x+v*v_y)-lambda2*(v_xx+v_yy))
c3 = sn.Data(u_x + v_y)

Keras <= 1.4.0 requires training on every target. Therefore, it will through an error if we do not train on $\psi$.

To resolve this error, let's add a trivial target for $\psi$:

In [ ]:
c4 = Psi*0.0

Define the optimization model with `sn.SciModel`:

In [ ]:
# Define the optimization model (set of inputs and constraints)
model = sn.SciModel(
    inputs=[x, y, t],
    targets=[d1, d2, d3, c1, c2, c3, c4],
    loss_func="mse",
    plot_to_file='NS-Model.png'
)

The network is plotted in the NS-Model.png file.

![SciModel](https://github.com/ianbgroves/sciann-applications/blob/master/SciANN-NavierStokes/NS-Model.png?raw=1)

Prepare the training data according to the order they are defined in `sn.SciModel`.

In [ ]:
input_data = [x_train, y_train, t_train]

In [ ]:
data_d1 = u_train
data_d2 = v_train
data_d3 = p_train
data_c1 = 'zeros'
data_c2 = 'zeros'
data_c3 = 'zeros'
data_c4 = 'zeros'
target_data = [data_d1, data_d2, data_d3, data_c1, data_c2, data_c3, data_c4]

Train the model by calling `.train` function. Check the documentation at www.sciann.com for all the training options.

In [ ]:
from time import time
start = time()
history = model.train(
    x_true=input_data,
    y_true=target_data,
    epochs=10000,
    batch_size=100,
    shuffle=True,
    learning_rate=0.001,
    reduce_lr_after=100,
    stop_loss_value=1e-8,
    verbose=1
)
print("time taken to train: {}s".format(time()-start))
model.save_weights('trained-navier-stokes.hdf5')


Total samples: 5000 
Batch size: 100 
Total batches: 50 

Epoch 1/10000
50/50 [==============================] - 33s 136ms/step - batch: 24.5000 - size: 100.0000 - loss: 0.4518 - Grad__loss: 0.3258 - mul_loss: 0.0910 - P_loss: 0.0205 - sub_2_loss: 0.0085 - sub_4_loss: 0.0060 - add_7_loss: 1.9559e-33 - mul_12_loss: 0.0000e+00 - lr: 0.0010 - time: 32.5576
Epoch 2/10000
50/50 [==============================] - 6s 129ms/step - batch: 24.5000 - size: 100.0000 - loss: 0.2173 - Grad__loss: 0.1150 - mul_loss: 0.0851 - P_loss: 0.0085 - sub_2_loss: 0.0061 - sub_4_loss: 0.0025 - add_7_loss: 2.4546e-33 - mul_12_loss: 0.0000e+00 - lr: 0.0010 - time: 6.4584
Epoch 3/10000
50/50 [==============================] - 6s 130ms/step - batch: 24.5000 - size: 100.0000 - loss: 0.1598 - Grad__loss: 0.0667 - mul_loss: 0.0788 - P_loss: 0.0082 - sub_2_loss: 0.0040 - sub_4_loss: 0.0021 - add_7_loss: 2.9342e-33 - mul_12_loss: 0.0000e+00 - lr: 0.0010 - time: 6.4805
Epoch 4/10000
50/50 [==============================

KeyboardInterrupt: ignored

In [ ]:
print("lambda1: {},  lambda2: {}".format(lambda1.value, lambda2.value))

In [ ]:
plt.semilogy(history.history['loss'])
plt.xlabel('epochs')
plt.ylabel('loss')